# DataFrames: Read and Store Data

<img src="../.images/dask-dataframe.svg" 
     align="right"
     width="20%"
     alt="Dask dataframes are blocked Pandas dataframes">
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

## Create local Dask Cluster

In [ ]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [ ]:
import dask.dataframe as dd

df = dd.demo.make_timeseries(start='2000-01-01', end='2000-01-31', freq='1s', partition_freq='1d',
                             dtypes={'name': str, 'id': int, 'x': float, 'y': float})
df

In [ ]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory.  Each of them holds some timeseries data.

In [ ]:
!ls data/*.csv | head

In [ ]:
!head data/2000-01-01.csv

In [ ]:
!head data/2000-01-30.csv

We can read many of these files with the dask.dataframe.read_csv function

In [ ]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

In [ ]:
df.head()

## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [ ]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [ ]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [ ]:
%time df.groupby('name').x.mean().compute()

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.  

We'll install the `pyarrow` library to help us read and write from Parquet

In [ ]:
!pip install pyarrow

In [ ]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [ ]:
!ls data/2000-01.parquet/

## Read from Parquet

In [ ]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

In [ ]:
%time df.groupby('name').x.mean().compute()

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [ ]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

Here the difference is not that large, but with larger datasets this can save a great deal of time.